In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
month = 4
year = 2024
urls = [f'https://tsn.ua/news?day={i}&month={month}&year={year}' for i in range(1, 31)]

In [4]:
def parse_html(html):
  soup = BeautifulSoup(html, 'html.parser')
  a_tag = soup.find('a', class_='c-card__link')
  title = a_tag.text.strip() if a_tag else None
  href = a_tag['href'] if a_tag else None
  time_tag = soup.find('time')
  datetime = time_tag['datetime'] if time_tag else None
  dd_tag = soup.find('dd', class_='c-bar__label i-before i-before--spacer-r-sm i-views')
  views = dd_tag.text.strip() if dd_tag else None
  return { 'title': title, 'href': href, 'datetime': datetime, 'views': views }

In [5]:
def make_request(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')
  divs = soup.find_all('div', class_='c-card__body')
  return [parse_html(str(div)) for div in divs]

In [8]:
news = [new for url in urls for new in make_request(url)]

In [9]:
df = pd.DataFrame(news)
df.head()
df.to_csv('news.csv')